In [1]:
import os
import pandas as pd
import cv2
import numpy as np
from tqdm import tqdm

def colour_quantisation(image, k=20):
    pixels = np.float32(image.reshape(-1, 3))
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _, labels, palette = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    quantised = np.uint8(palette)[labels.flatten()]
    quantised = quantised.reshape(image.shape)
    return quantised

def preprocess_images(csv_file, img_dir, output_dir, is_test=False, image_size=224, zoom_proportion=0.15, quantize_k=20):
    os.makedirs(output_dir, exist_ok=True)
    quantized_dir = os.path.join(output_dir, 'quantized')
    os.makedirs(quantized_dir, exist_ok=True)

    df = pd.read_csv(csv_file)

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing images"):
        img_name = str(row[0])
        img_path = os.path.join(img_dir, img_name)
        if not os.path.exists(img_path):
            img_path = os.path.join(img_dir, f"{img_name}.jpg")

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (image_size, image_size), interpolation=cv2.INTER_LINEAR)

        # Save original and quantized versions
        cv2.imwrite(os.path.join(output_dir, f"{img_name}_original.jpg"), cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
        quantized = colour_quantisation(image, k=quantize_k)
        cv2.imwrite(os.path.join(quantized_dir, f"{img_name}_quantized.jpg"), cv2.cvtColor(quantized, cv2.COLOR_RGB2BGR))

        if not is_test:
            # Create augmented versions only for training images
            # Flipped
            flipped = cv2.flip(image, 1)
            cv2.imwrite(os.path.join(output_dir, f"{img_name}_flipped.jpg"), cv2.cvtColor(flipped, cv2.COLOR_RGB2BGR))
            flipped_quantized = colour_quantisation(flipped, k=quantize_k)
            cv2.imwrite(os.path.join(quantized_dir, f"{img_name}_flipped_quantized.jpg"), cv2.cvtColor(flipped_quantized, cv2.COLOR_RGB2BGR))

            # Zoomed
            height, width = image.shape[:2]
            crop_size = int(min(width, height) * (1 - zoom_proportion))
            start_x = (width - crop_size) // 2
            start_y = (height - crop_size) // 2
            zoomed = image[start_y:start_y+crop_size, start_x:start_x+crop_size]
            zoomed = cv2.resize(zoomed, (image_size, image_size), interpolation=cv2.INTER_LINEAR)
            cv2.imwrite(os.path.join(output_dir, f"{img_name}_zoomed.jpg"), cv2.cvtColor(zoomed, cv2.COLOR_RGB2BGR))
            zoomed_quantized = colour_quantisation(zoomed, k=quantize_k)
            cv2.imwrite(os.path.join(quantized_dir, f"{img_name}_zoomed_quantized.jpg"), cv2.cvtColor(zoomed_quantized, cv2.COLOR_RGB2BGR))

            # Zoomed and flipped
            zoomed_flipped = cv2.flip(zoomed, 1)
            cv2.imwrite(os.path.join(output_dir, f"{img_name}_zoomed_flipped.jpg"), cv2.cvtColor(zoomed_flipped, cv2.COLOR_RGB2BGR))
            zoomed_flipped_quantized = colour_quantisation(zoomed_flipped, k=quantize_k)
            cv2.imwrite(os.path.join(quantized_dir, f"{img_name}_zoomed_flipped_quantized.jpg"), cv2.cvtColor(zoomed_flipped_quantized, cv2.COLOR_RGB2BGR))

    print(f"Preprocessing complete. Images saved to {output_dir}")
    print(f"Quantized images saved to {quantized_dir}")

# Example usage
if __name__ == "__main__":
    config = {
        'train_csv': './COMP90086_2024_Project_train/train.csv',
        'train_img_dir': './COMP90086_2024_Project_train/train',
        'train_output_dir': './preprocessed_images/train',
        'test_csv': './COMP90086_2024_Project_test/test.csv',
        'test_img_dir': './COMP90086_2024_Project_test/test',
        'test_output_dir': './preprocessed_images/test',
        'image_size': 224,
        'zoom_proportion': 0.15,
        'quantize_k': 20
    }

    # Process training images
    preprocess_images(
        csv_file=config['train_csv'],
        img_dir=config['train_img_dir'],
        output_dir=config['train_output_dir'],
        is_test=False,
        image_size=config['image_size'],
        zoom_proportion=config['zoom_proportion'],
        quantize_k=config['quantize_k']
    )

    # Process test images
    preprocess_images(
        csv_file=config['test_csv'],
        img_dir=config['test_img_dir'],
        output_dir=config['test_output_dir'],
        is_test=True,
        image_size=config['image_size'],
        zoom_proportion=config['zoom_proportion'],
        quantize_k=config['quantize_k']
    )

Processing images:   0%|          | 0/7680 [00:00<?, ?it/s]/tmp/ipykernel_7842/848516407.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  img_name = str(row[0])
Processing images: 100%|██████████| 7680/7680 [47:41<00:00,  2.68it/s]


Preprocessing complete. Images saved to ./preprocessed_images/train
Quantized images saved to ./preprocessed_images/train/quantized


Processing images:   0%|          | 0/1920 [00:00<?, ?it/s]/tmp/ipykernel_7842/848516407.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  img_name = str(row[0])
Processing images: 100%|██████████| 1920/1920 [03:03<00:00, 10.49it/s]

Preprocessing complete. Images saved to ./preprocessed_images/test
Quantized images saved to ./preprocessed_images/test/quantized
